In [2]:
from Bio import SeqIO as io
from tqdm import tqdm
import csv
import pandas as pd
from Bio import pairwise2 as pw2
import random
import numpy as np

In [3]:
#Preparing a dataframe for the RFAM records
recs = []
for rec in tqdm(io.parse("rfam.fasta", "fasta")):
    headers = rec.description.split(maxsplit=1)
    seqid = headers[0]
    desc = headers[1] if len(headers) > 1 else ""
    seq = rec.seq

    if len(seq) > 299 and len(seq) < 1001:
        recs.append({
            "sequence_id": seqid,#.split("_")[0],
            "description": desc,
            "sequence": str(seq)
        })
records = pd.DataFrame(recs)

7818286it [00:12, 620322.05it/s]


In [4]:
#Preparing a dataframe for the sequence mapping annotations
seqmap = pd.read_csv("sequence_rfam_mapping_annotations.csv")
families = pd.unique(seqmap['rfam_id'])

In [5]:
#Preparing a dataframe with the rfam families and sequence_ids with species names
df = pd.concat([records["sequence_id"], seqmap["rfam_id"]], axis = 1)

In [6]:
#Preparing a function that fetches sequence given number of sequence ids
def fetch_ids(family, df, n):

    assert(n <= df.loc[df["rfam_id"] == family].shape[0])
    
    seqs = df.loc[df["rfam_id"] == family]["sequence_id"].sample(n).values
    
    return(seqs.tolist())

In [9]:
#Creating long list of sequences to be aligned
ids = []
indices = []
n = 3
for i, family in tqdm(enumerate(families)):
    #Checking loop count and also avoiding "nan" families 
    #if i == 100: break   
    if isinstance(family, str) == False: continue
    
    #Setting sequence counts
    seqs = df.loc[df["rfam_id"] == family].shape[0]
    members = n if seqs > n else seqs
    #print("Family: " + family + " Members: " + str(members) + " Sequences: " + str(seqs))

    #Appending sequence ids
    for i in fetch_ids(family, df, members):
        ids.append(i)
        indices.append(records.loc[records["sequence_id"] == i].index.values[0])

190it [00:06, 31.13it/s]


In [10]:
#Performing pairwise alignments
huge = np.empty((len(ids), len(ids)))

for i, id1 in tqdm(enumerate(ids)):
    for j, id2 in enumerate(ids):
        if j >= i:
            seq1 = str(records.loc[records["sequence_id"] == id1].iat[0,2])
            seq2 = str(records.loc[records["sequence_id"] == id2].iat[0,2])
            alignment = pw2.align.globalxx(seq1,seq2)
            huge[i][j] = alignment[0][2]
        else:
            huge[i][j] = huge[j][i]
print("DONE!")

513it [49:20,  5.77s/it]

DONE!


In [11]:
huge_df = pd.DataFrame(huge, columns=indices)
huge_df = huge_df.set_axis(indices)
huge_df

,147699,123313,156785,32880,42722,78871,55002,14329,50856,40779,...,96054,97829,108588,115141,126429,132233,146599,150036,165502,168930
147699,763.0,495.0,420.0,307.0,312.0,361.0,318.0,295.0,318.0,363.0,...,280.0,315.0,300.0,277.0,296.0,292.0,309.0,310.0,295.0,539.0
123313,495.0,781.0,512.0,304.0,312.0,364.0,319.0,291.0,318.0,366.0,...,287.0,323.0,300.0,285.0,302.0,287.0,313.0,313.0,296.0,548.0
156785,420.0,512.0,575.0,270.0,286.0,309.0,279.0,270.0,283.0,318.0,...,246.0,281.0,266.0,243.0,274.0,256.0,288.0,277.0,258.0,468.0
32880,307.0,304.0,270.0,340.0,226.0,259.0,222.0,212.0,230.0,251.0,...,198.0,225.0,211.0,196.0,212.0,205.0,225.0,220.0,208.0,314.0
42722,312.0,312.0,286.0,226.0,367.0,258.0,213.0,210.0,229.0,238.0,...,190.0,226.0,206.0,173.0,213.0,210.0,231.0,220.0,188.0,338.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
132233,292.0,287.0,256.0,205.0,210.0,228.0,205.0,199.0,214.0,231.0,...,186.0,216.0,203.0,178.0,207.0,312.0,216.0,203.0,193.0,294.0
146599,309.0,313.0,288.0,225.0,231.0,245.0,222.0,225.0,236.0,250.0,...,198.0,231.0,218.0,187.0,231.0,216.0,356.0,226.0,201.0,331.0
150036,310.0,313.0,277.0,220.0,220.0,245.0,219.0,210.0,225.0,248.0,...,207.0,218.0,219.0,199.0,215.0,203.0,226.0,354.0,206.0,330.0
165502,295.0,296.0,258.0,208.0,188.0,242.0,217.0,192.0,218.0,244.0,...,200.0,199.0,209.0,205.0,202.0,193.0,201.0,206.0,321.0,297.0


In [12]:
huge_df.to_csv("alignment_matrix_"+ str(n) +"_perfam.csv")